In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from queue import Queue
from scipy import ndimage

In [ ]:
#加载数据
fpath1= "C:/Users/pc_207/Desktop/2023a_cm" + "/seg_nii/T2_3.nii.gz"
img = nib.load(fpath1)

#加载mask
fpath2= "C:/Users/pc_207/Desktop/2023a_cm" + "/seg_nii/T2_mask_3.nii"
mask = nib.load(fpath2)

#加载seg
fpath3= "C:/Users/pc_207/Desktop/2023a_cm" + "/seg_nii/T2_seg_mask_3.nii.gz"
seg_mask = nib.load(fpath3)

In [ ]:
width,height,queue=img.dataobj.shape#img.dataobj.shape[0:3]
pixdim = img.header["pixdim"]
print( "shape:",width,height,queue,"pixdim:",pixdim[1],pixdim[2],pixdim[3])

In [ ]:
def get_edge(edge,num,start,flag):
    temp=0
    num=num
    while temp==0:
        if flag == 0:
            temp = np.sum(edge[num,:,:])
        if flag == 1:
            temp = np.sum(edge[:,num,:])
        if flag == 2:
            temp = np.sum(edge[:,:,num])
        if start:
            num = num+1
        else:
            num = num-1
    if start:
        return num-1
    else:
        return num+1

e1 = get_edge(seg_mask.dataobj[:, :, :], 0, True,0)
e2 = get_edge(seg_mask.dataobj[:, :, :], 256-1, False,0)
e3 = get_edge(seg_mask.dataobj[:, :, :], 0, True,1)
e4 = get_edge(seg_mask.dataobj[:, :, :], 256-1, False,1)
e5 = get_edge(seg_mask.dataobj[:, :, :], 0, True,2)
e6 = get_edge(seg_mask.dataobj[:, :, :], 192-1, False,2)

In [ ]:
len_x,len_y,len_z = e2-e1,e4-e3,e6-e5
print(len_x,len_y,len_z)

img_arr = img.dataobj[e1:e2, e3:e4, e5:e6]#[:,:,:,0,0]
mask_arr = mask.dataobj[e1:e2, e3:e4, e5:e6]
seg_mask_arr = seg_mask.dataobj[e1:e2, e3:e4, e5:e6]/2

#先减去再生长
img_arr = img_arr*(1-mask_arr)

In [ ]:
#plt.imshow(seg_mask_arr[15,:,:],cmap="gray")

In [ ]:
def grow_seg(image, seed, threshold,th2=0.5):
                            
    height, width, depth = image.shape
    segmented = np.zeros_like(image)
    visited = np.zeros_like(image)
    
    q = Queue()
    q.put(seed)
    region_sum = 0
    region_size = 0
    
    while not q.empty():
        current = q.get()
        x, y, z = current
        if visited[x, y, z]:
            continue
        
        visited[x, y, z] = 1
        if np.abs(image[x, y, z] - image[seed]) <= threshold:
            segmented[x, y, z] = 1
            region_sum += image[x, y, z]
            region_size += 1
            
            for i in range(-1, 2):
                for j in range(-1, 2):
                    for k in range(-1, 2):
                        if 0 <= x+i < height and 0 <= y+j < width and 0 <= z+k < depth:
                            if np.abs(image[x+i, y+j, z+k] - image[x, y, z]) <= 0.02:#这里是我后加的
                                q.put((x+i, y+j, z+k))
                                
    
    for i in range(segmented.shape[2]):
        image = segmented[:,:,i]
        labeled_image, num_features = ndimage.label(image)

        
        sizes = ndimage.sum(image, labeled_image, range(1, num_features + 1))

       
        max_label = np.argmax(sizes) + 1

        
        largest_component = (labeled_image == max_label).astype(np.uint8)
        segmented[:,:,i] = largest_component
    for i in range(segmented.shape[0]):
        image = segmented[i,:,:]
        labeled_image, num_features = ndimage.label(image)

       
        sizes = ndimage.sum(image, labeled_image, range(1, num_features + 1))

        
        max_label = np.argmax(sizes) + 1

        
        largest_component = (labeled_image == max_label).astype(np.uint8)
        segmented[i,:,:] = largest_component
    for i in range(segmented.shape[1]):
        image = segmented[:,i,:]
        labeled_image, num_features = ndimage.label(image)

        
        sizes = ndimage.sum(image, labeled_image, range(1, num_features + 1))

        
        max_label = np.argmax(sizes) + 1

        
        largest_component = (labeled_image == max_label).astype(np.uint8)
        segmented[:,i,:] = largest_component
    
    return segmented

img_arr = img_arr-np.min(img_arr)
img_arr = img_arr/np.max(img_arr)
seg_arr = grow_seg(img_arr,(int(len_x/2),int(len_y/2),int(len_z/2)),0.1)

In [ ]:
for i in range(len_z):
    
    plt.subplot(1,3,1)
    plt.imshow(seg_arr[:,:,i],cmap="gray")
    
    plt.subplot(1,3,2)
    plt.imshow(seg_mask_arr[:,:,i],cmap="gray")
    
    plt.subplot(1,3,3)
    plt.imshow( np.abs(seg_arr[:,:,i]-seg_mask_arr[:,:,i]),cmap="gray")
    plt.show()
    input()